# Imports

In [ ]:
# import packages for reading in data
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

# import packages for types
from pyspark.sql.types import IntegerType, DoubleType

# import functions for getting substring & rounding numbers
from pyspark.sql.functions import substring, round

# import functions for model
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# import packages for turning the recommendations into seperate columns
from pyspark.sql.functions import col, explode

# import packages for interactively selecting user
from ipywidgets import widgets, interact


# Read in the data

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ALS Recommender System") \
    .master("local[*]") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/05/08 10:28:09 WARN Utils: Your hostname, Gloria resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/08 10:28:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/08 10:28:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
ratings = spark.read.csv("file:///home/moraa/Spring-2025/CSE817/ml-latest-small/ratings.csv", header=True, inferSchema=True)


In [7]:
ratings.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [8]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [9]:
ratings.count()

100836

In [10]:
movies = spark.read.csv("file:///home/moraa/Spring-2025/CSE817/ml-latest-small/movies.csv", header=True, inferSchema=True)

In [11]:
movies.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [12]:
from pyspark.sql.functions import regexp_extract

movies = movies.withColumn("year", regexp_extract("title", r'\\((\\d{4})\\)', 1))

In [13]:
ratings = ratings.withColumnRenamed("userId", "user_id") \
                 .withColumnRenamed("movieId", "movie_id")

movies = movies.withColumnRenamed("movieId", "movie_id")


In [14]:
combined = ratings.join(movies, on="movie_id", how="inner")

In [15]:
combined.printSchema()
combined.show(5)

root
 |-- movie_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- year: string (nullable = true)

+--------+-------+------+---------+--------------------+--------------------+----+
|movie_id|user_id|rating|timestamp|               title|              genres|year|
+--------+-------+------+---------+--------------------+--------------------+----+
|       1|      1|   4.0|964982703|    Toy Story (1995)|Adventure|Animati...|    |
|       3|      1|   4.0|964981247|Grumpier Old Men ...|      Comedy|Romance|    |
|       6|      1|   4.0|964982224|         Heat (1995)|Action|Crime|Thri...|    |
|      47|      1|   5.0|964983815|Seven (a.k.a. Se7...|    Mystery|Thriller|    |
|      50|      1|   5.0|964982931|Usual Suspects, T...|Crime|Mystery|Thr...|    |
+--------+-------+------+---------+--------------------+--

In [16]:
df = combined

In [17]:
df.printSchema()
df.show(5)

root
 |-- movie_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- year: string (nullable = true)

+--------+-------+------+---------+--------------------+--------------------+----+
|movie_id|user_id|rating|timestamp|               title|              genres|year|
+--------+-------+------+---------+--------------------+--------------------+----+
|       1|      1|   4.0|964982703|    Toy Story (1995)|Adventure|Animati...|    |
|       3|      1|   4.0|964981247|Grumpier Old Men ...|      Comedy|Romance|    |
|       6|      1|   4.0|964982224|         Heat (1995)|Action|Crime|Thri...|    |
|      47|      1|   5.0|964983815|Seven (a.k.a. Se7...|    Mystery|Thriller|    |
|      50|      1|   5.0|964982931|Usual Suspects, T...|Crime|Mystery|Thr...|    |
+--------+-------+------+---------+--------------------+--

In [19]:
df = df.drop("timestamp")

In [20]:
df.show(10, truncate=False)

+--------+-------+------+---------------------------+-------------------------------------------+----+
|movie_id|user_id|rating|title                      |genres                                     |year|
+--------+-------+------+---------------------------+-------------------------------------------+----+
|1       |1      |4.0   |Toy Story (1995)           |Adventure|Animation|Children|Comedy|Fantasy|    |
|3       |1      |4.0   |Grumpier Old Men (1995)    |Comedy|Romance                             |    |
|6       |1      |4.0   |Heat (1995)                |Action|Crime|Thriller                      |    |
|47      |1      |5.0   |Seven (a.k.a. Se7en) (1995)|Mystery|Thriller                           |    |
|50      |1      |5.0   |Usual Suspects, The (1995) |Crime|Mystery|Thriller                     |    |
|70      |1      |3.0   |From Dusk Till Dawn (1996) |Action|Comedy|Horror|Thriller              |    |
|101     |1      |5.0   |Bottle Rocket (1996)       |Adventure|Comedy|Cri

In [22]:
from pyspark.sql.functions import regexp_extract

df = df.withColumn("year", regexp_extract("title", r"\((\d{4})\)", 1))

In [23]:
df.select("title", "year").show(10, truncate=False)

+---------------------------+----+
|title                      |year|
+---------------------------+----+
|Toy Story (1995)           |1995|
|Grumpier Old Men (1995)    |1995|
|Heat (1995)                |1995|
|Seven (a.k.a. Se7en) (1995)|1995|
|Usual Suspects, The (1995) |1995|
|From Dusk Till Dawn (1996) |1996|
|Bottle Rocket (1996)       |1996|
|Braveheart (1995)          |1995|
|Rob Roy (1995)             |1995|
|Canadian Bacon (1995)      |1995|
+---------------------------+----+
only showing top 10 rows



In [24]:
from pyspark.sql.functions import regexp_replace

df = df.withColumn(
    "title",
    regexp_replace("title", r"\s*\(\d{4}\)", "")
)


In [25]:
df.select("title", "year").show(10, truncate=False)

+--------------------+----+
|title               |year|
+--------------------+----+
|Toy Story           |1995|
|Grumpier Old Men    |1995|
|Heat                |1995|
|Seven (a.k.a. Se7en)|1995|
|Usual Suspects, The |1995|
|From Dusk Till Dawn |1996|
|Bottle Rocket       |1996|
|Braveheart          |1995|
|Rob Roy             |1995|
|Canadian Bacon      |1995|
+--------------------+----+
only showing top 10 rows



In [26]:
df.show(10, truncate=False)

+--------+-------+------+--------------------+-------------------------------------------+----+
|movie_id|user_id|rating|title               |genres                                     |year|
+--------+-------+------+--------------------+-------------------------------------------+----+
|1       |1      |4.0   |Toy Story           |Adventure|Animation|Children|Comedy|Fantasy|1995|
|3       |1      |4.0   |Grumpier Old Men    |Comedy|Romance                             |1995|
|6       |1      |4.0   |Heat                |Action|Crime|Thriller                      |1995|
|47      |1      |5.0   |Seven (a.k.a. Se7en)|Mystery|Thriller                           |1995|
|50      |1      |5.0   |Usual Suspects, The |Crime|Mystery|Thriller                     |1995|
|70      |1      |3.0   |From Dusk Till Dawn |Action|Comedy|Horror|Thriller              |1996|
|101     |1      |5.0   |Bottle Rocket       |Adventure|Comedy|Crime|Romance             |1996|
|110     |1      |4.0   |Braveheart     

See how many users and movies there are

In [28]:
print("There are", df.select("user_id").dropDuplicates().count(), "users in this dataset.")

There are 610 users in this dataset.


In [29]:
print("There are", df.select("movie_id").dropDuplicates().count(), "movies in this dataset.")

There are 9724 movies in this dataset.


See what their max and min values are

In [31]:
print ("Min user_id =", df.agg({"user_id": "min"}).collect()[0])
print ("Max user_id =", df.agg({"user_id": "max"}).collect()[0])

Min user_id = Row(min(user_id)=1)
Max user_id = Row(max(user_id)=610)


In [32]:
print ("Min movie_id =", df.agg({"movie_id": "min"}).collect()[0])
print ("Max movie_id =", df.agg({"movie_id": "max"}).collect()[0])

Min movie_id = Row(min(movie_id)=1)
Max movie_id = Row(max(movie_id)=193609)


In [33]:
df.select(["movie_id", "title"]).drop_duplicates().show(10, truncate=False)


+--------+--------------------------------------------------------+
|movie_id|title                                                   |
+--------+--------------------------------------------------------+
|1136    |Monty Python and the Holy Grail                         |
|775     |Spirits of the Dead                                     |
|848     |Spitfire Grill, The                                     |
|1882    |Godzilla                                                |
|48774   |Children of Men                                         |
|158872  |Sausage Party                                           |
|60040   |Incredible Hulk, The                                    |
|73321   |Book of Eli, The                                        |
|99728   |Gangster Squad                                          |
|29      |City of Lost Children, The (Cité des enfants perdus, La)|
+--------+--------------------------------------------------------+
only showing top 10 rows



Check if there are users with multiple ratings on a particular movie

In [34]:
if df["movie_id", "user_id"].count() > df.dropDuplicates(["movie_id", "user_id"]).count():
    raise ValueError('Data has duplicates')

## Calculate the sparcity of a users vs movies matrix

In [35]:
# Count the total number of ratings in the dataset
num_ratings = float(df.select("rating").count())

# Count the number of distinct userIds and distinct movieIds
num_users = df.select("user_id").distinct().count()
num_movies = df.select("movie_id").distinct().count()

In [37]:
# Caluclate the sparsity
sparsity = (1.0 - (num_ratings / (num_users * num_movies))) * 100

print ("The ratings dataframe is %.2f" % sparsity + "% empty.")

The ratings dataframe is 98.30% empty.


## Build recommender model

Split the data into a train & test split

In [38]:
# Split the data to create a test and train set
train, test = df.randomSplit([0.8, 0.2], seed = 42)

Create model

In [39]:
# Create ALS model
als = ALS(userCol="user_id", itemCol="movie_id", ratingCol="rating", nonnegative=True, implicitPrefs=False, coldStartStrategy="drop")

Fit the model to the trainind data

In [40]:
# train out model by fitting it to the training data
model = als.fit(train)

Evaluate the model

In [41]:
# run the test set through the model to get a prediction
unrounded_predictions = model.transform(test)

In [42]:
# round the predictions
predictions = unrounded_predictions.withColumn("prediction", round(unrounded_predictions["prediction"], 3))

In [43]:
# Compare the user rating to the predicted rating
predictions.show(10)

+--------+-------+------+--------------------+--------------------+----+----------+
|movie_id|user_id|rating|               title|              genres|year|prediction|
+--------+-------+------+--------------------+--------------------+----+----------+
|     296|    471|   4.0|        Pulp Fiction|Comedy|Crime|Dram...|1994|     4.108|
|     356|    463|   4.0|        Forrest Gump|Comedy|Drama|Roma...|1994|     3.998|
|    1197|    148|   3.0| Princess Bride, The|Action|Adventure|...|1987|     4.086|
|    1221|    463|   4.5|Godfather: Part I...|         Crime|Drama|1974|     4.035|
|    1690|    463|   4.0| Alien: Resurrection|Action|Horror|Sci-Fi|1997|      3.28|
|    2167|    463|   3.0|               Blade|Action|Horror|Thr...|1998|     3.706|
|    2394|    496|   3.5|Prince of Egypt, The|   Animation|Musical|1998|     3.125|
|    4191|    496|   2.5|               Alfie|Comedy|Drama|Romance|1966|      4.07|
|    4223|    463|   4.0|  Enemy at the Gates|           Drama|War|2001|    

We will now attempt to get an official metric on the accuracy of these predictions.

In [44]:
# Define an evaluator using RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [45]:
# evaluate model performance
rmse = evaluator.evaluate(predictions)

print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.8754690602840302


## Recommend movies

1. Generate recommendation for all users


In [46]:
num_movies_to_recommend = 5

In [47]:
# Generate n Recommendations for all users
recommendations_for_users = model.recommendForAllUsers(num_movies_to_recommend)

In [48]:
recommendations_for_users.limit(10).show(truncate=False)

+-------+--------------------------------------------------------------------------------------------------+
|user_id|recommendations                                                                                   |
+-------+--------------------------------------------------------------------------------------------------+
|1      |[{132333, 5.7711916}, {5490, 5.7711916}, {5915, 5.7511315}, {945, 5.747388}, {3983, 5.6669397}]   |
|2      |[{1218, 5.021182}, {131724, 4.892015}, {7842, 4.867265}, {60943, 4.8495092}, {59018, 4.8495092}]  |
|3      |[{1241, 5.3868766}, {89118, 5.2076526}, {2068, 4.947723}, {6835, 4.9265428}, {7982, 4.918167}]    |
|4      |[{25825, 5.654018}, {48322, 5.5147}, {25850, 5.3838487}, {926, 5.0376754}, {74452, 4.955408}]     |
|5      |[{132333, 4.8848243}, {5490, 4.8848243}, {5915, 4.833097}, {3379, 4.7773905}, {6375, 4.7637954}]  |
|6      |[{2563, 5.086253}, {7121, 4.836037}, {3404, 4.823554}, {1066, 4.797053}, {42730, 4.795869}]       |
|7      |[{5222, 5.

We can turn this recommendations column into two columns for easier visualisation of the recommended movies. This gves the following:



In [49]:
# turn the recommendation column into multiple columns for easy visualisation
user_rec_nd_unrounded_pred = recommendations_for_users.withColumn("exploded_rec", explode("recommendations"))\
                                                    .select('user_id', col("exploded_rec.movie_id"), col("exploded_rec.rating"))

In [50]:
# round the predicted ratings
user_rec_nd_pred_rating = user_rec_nd_unrounded_pred.withColumn("rating", round(user_rec_nd_unrounded_pred["rating"], 3))


In [51]:
user_rec_nd_pred_rating.limit(10).show()

+-------+--------+------+
|user_id|movie_id|rating|
+-------+--------+------+
|      1|  132333| 5.771|
|      1|    5490| 5.771|
|      1|    5915| 5.751|
|      1|     945| 5.747|
|      1|    3983| 5.667|
|      2|    1218| 5.021|
|      2|  131724| 4.892|
|      2|    7842| 4.867|
|      2|   60943|  4.85|
|      2|   59018|  4.85|
+-------+--------+------+



Each user now has 5 recommended movies assigned to them by the recommender system. These movies are based of a predicted rating of what each user would rate each movie and the top 'n' predicted rated movies for each user are chosen.

We can further dive into a specific user and see **how the recommendations compare to the users actual highly rated movies.**

The example below follows the user with user_id of 12.

- Users top rated movies


In [52]:
df.select(df.user_id, df.movie_id, df.title, df.year, df.genres, df.rating) \
  .filter('user_id = 12') \
  .sort('rating', ascending=False).limit(10).show(truncate=False)

+-------+--------+--------------------------+----+--------------------+------+
|user_id|movie_id|title                     |year|genres              |rating|
+-------+--------+--------------------------+----+--------------------+------+
|12     |168     |First Knight              |1995|Action|Drama|Romance|5.0   |
|12     |222     |Circle of Friends         |1995|Drama|Romance       |5.0   |
|12     |256     |Junior                    |1994|Comedy|Sci-Fi       |5.0   |
|12     |838     |Emma                      |1996|Comedy|Drama|Romance|5.0   |
|12     |2581    |Never Been Kissed         |1999|Comedy|Romance      |5.0   |
|12     |1357    |Shine                     |1996|Drama|Romance       |5.0   |
|12     |1721    |Titanic                   |1997|Drama|Romance       |5.0   |
|12     |2072    |'burbs, The               |1989|Comedy              |5.0   |
|12     |2485    |She's All That            |1999|Comedy|Romance      |5.0   |
|12     |2572    |10 Things I Hate About You|1999|Co

- Recommended movies

In [53]:
user_rec_nd_pred_rating.join(df.drop('user_id', 'rating').dropDuplicates(), ['movie_id'], how='left') \
                       .select(col('user_id'), col('movie_id'), col('title'), col('year'), col('genres'), col('rating')) \
                       .filter('user_id = 12') \
                       .sort('rating', ascending=False).limit(10).show(truncate=False)

+-------+--------+------------------+----+-------------------------------+------+
|user_id|movie_id|title             |year|genres                         |rating|
+-------+--------+------------------+----+-------------------------------+------+
|12     |7121    |Adam's Rib        |1949|Comedy|Romance                 |6.007 |
|12     |2563    |Dangerous Beauty  |1998|Drama                          |5.987 |
|12     |67618   |Strictly Sexual   |2008|Comedy|Drama|Romance           |5.927 |
|12     |33649   |Saving Face       |2004|Comedy|Drama|Romance           |5.923 |
|12     |1011    |Herbie Rides Again|1974|Children|Comedy|Fantasy|Romance|5.833 |
+-------+--------+------------------+----+-------------------------------+------+



Here we have shown the users top 10 rated movies and the 5 movies that the system recommends for this given user. As you can see from the users highly rated movies, the genres of these movies are most commonly Drama, Romance and Comody. It can also be seen that our recommendation system has taken this on board and has recommended movies with the genres Comody, Drama and Romance.

This shows the effectiveness of the recommender system's recommendations.